In [107]:
!pip install pandas folium geopandas geopy datacommons_pandas sodapy

In [108]:
import pandas as pd
import matplotlib as mpl
#import seaborn as sea
#import folium as fol
import sklearn as sci
import requests
import folium
import datacommons_pandas as dc

from geopy.geocoders import Nominatim
from tqdm import tqdm
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster

In [109]:
api_url = "https://data.fortworthtexas.gov/resource/k6ic-7kp7.csv" # Replace with actual API endpoint

response = requests.get(api_url) # For GET request
# response = requests.post(api_url, data=payload) # For POST request with data

csv_filename = "fort_worth_crime_data.csv"

with open(csv_filename, "wb") as file:
    file.write(response.content)

# Read CSV into a DataFrame
df = pd.read_csv(csv_filename)

# Display the first few rows
df.head()


,case_no_offense,case_no,reported_date,nature_of_call,from_date,offense,offense_desc,block_address,city,state,beat,division,councildistrict,attempt_complete,location_type,locationtypedescription,location_1
0,180094593-35A,180094593,2018-10-22T18:15:00.000,NARCOTIC VIOLATIONS,2018-10-22T18:15:00.000,35A,HSC 481.115(B) Poss CS PG 1 <1G 35A DRUG/NARCOTIC,700 CALIFORNIA PKWY N WB,FORT WORTH,TX,J13,NaN,NaN,C,13.0,13 HIGHWAY/ROAD/ALLEY,NaN
1,200085517-90Z,200085517,2020-11-03T13:24:00.000,HARASS,2020-08-12T05:18:00.000,90Z,PC 42.07(C) Harassment,1300 MARLBOROUGH DR,FORT WORTH,TX,I16,South,8.0,C,20.0,20 RESIDENCE/HOME,"\n, \n(32.640426597727426, -97.33851074771222)"
2,210012743-240,210012743,2021-02-20T20:27:59.000,AUTO THEFT,2021-02-20T20:27:59.000,240,PC 31.03(E)(4)(A)AUTO Theft>$1500<$20K (Auto t...,SANDY LN & EDERVILLE RD,FORT WORTH,TX,H15,East,5.0,C,18.0,18 PARKING LOT/GARAGE,NaN
3,190047946-35A,190047946,2019-06-04T01:33:00.000,NARCOTIC VIOLATIONS,2019-06-04T01:33:49.000,35A,HSC 481.117(B) Poss CS PG 3 <28G 35A DRUG/NARC...,OAK GROVE RD / E IH 20 FWY EB,NaN,TX,I17,NaN,NaN,C,13.0,13 HIGHWAY/ROAD/ALLEY,NaN
4,190031427-90D,190031427,2019-04-13T02:44:00.000,DWI,2019-04-13T02:44:37.000,90D,PC 49.04 Driving While Intoxicated 90D DRIVING...,WEST FWY SR WB / GREEN OAKS RD,NaN,TX,L11,NaN,NaN,C,13.0,13 HIGHWAY/ROAD/ALLEY,NaN


In [110]:
division_counts = df["division"].value_counts()
print(division_counts)

division
East         144
South        110
West         104
Northwest     69
Central       67
North         49
Name: count, dtype: int64


In [111]:
fort_worth_map = folium.Map(location=[32.7555, -97.3308], zoom_start=11)

# Show the map
fort_worth_map

In [112]:
geolocator = Nominatim(user_agent="fort_worth_geocoder")

# Function to get latitude and longitude from an address
def get_lat_long(address):
    try:
        location = geolocator.geocode(address + ", Fort Worth, TX")
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except:
        return None, None

# Apply geocoding to a sample (to avoid rate limits, use a subset)
tqdm.pandas()
df[["latitude", "longitude"]] = df["block_address"].progress_apply(lambda x: pd.Series(get_lat_long(str(x))))

# Drop rows with missing coordinates
df = df.dropna(subset=["latitude", "longitude"])

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [28:23<00:00,  1.70s/it]


In [113]:

marker_cluster = MarkerCluster().add_to(fort_worth_map)

# Plot each crime location
for _, row in df.iterrows():
    folium.Marker(location=[row["latitude"], row["longitude"]],
                  popup=row["block_address"]).add_to(marker_cluster)

# Show the map
fort_worth_map

# # Create a base map centered in Fort Worth
# fort_worth_map = folium.Map(location=[32.7555, -97.3308], zoom_start=11)

# # Add crime heatmap layer
# heat_data = df[["latitude", "longitude"]].values.tolist()
# HeatMap(heat_data).add_to(fort_worth_map)

# # Show the map
# fort_worth_map


In [205]:
API_KEY = "kOD-SoCJviK3JT-7j-p6mb5-k6b8yISrvdAIhjPfzFTz4TA_ivqQy0SO-t5nk1COe-jY8zdRL63GlYVPoleEP3Y3JM67iLl0GWW_gdRdXvqWB36BUBOROZMpQjCwZ3Yx"  # Replace with your Yelp API key
url = "https://api.yelp.com/v3/businesses/search"

headers = {"Authorization": f"Bearer {API_KEY}"}
params = {"term": "Italian", "location": "Fort Worth, TX", "categories": "restaurants", "limit": 50}

response = requests.get(url, headers=headers, params=params)
data = response.json()

# Print restaurant names and addresses
restaurants = []
for business in data["businesses"]:
    name = business["name"]
    lat = business["coordinates"]["latitude"]
    lon = business["coordinates"]["longitude"]
    rating = business["rating"]
    address = ", ".join(business["location"]["display_address"])
    restaurants.append((name, lat, lon, rating, address))

# Check if data was fetched correctly
print(f"Fetched {len(restaurants)} restaurants.")


Fetched 50 restaurants.


In [115]:
fort_worth_italian_rest_map = folium.Map(location=[32.7555, -97.3308], zoom_start=12)

# Add markers for each restaurant
for name, lat, lon, rating, address in restaurants:
    popup_text = f"<b>{name}</b><br>Rating: {rating} ⭐<br>{address}"
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color="red", icon="cutlery", prefix="fa"),
    ).add_to(fort_worth_italian_rest_map)


# Display the map
fort_worth_italian_rest_map


In [117]:
# Check column names in the df DataFrame
print(df.columns)


Index(['case_no_offense', 'case_no', 'reported_date', 'nature_of_call',
       'from_date', 'offense', 'offense_desc', 'block_address', 'city',
       'state', 'beat', 'division', 'councildistrict', 'attempt_complete',
       'location_type', 'locationtypedescription', 'location_1', 'latitude',
       'longitude'],
      dtype='object')


In [133]:
API_KEY = "kOD-SoCJviK3JT-7j-p6mb5-k6b8yISrvdAIhjPfzFTz4TA_ivqQy0SO-t5nk1COe-jY8zdRL63GlYVPoleEP3Y3JM67iLl0GWW_gdRdXvqWB36BUBOROZMpQjCwZ3Yx"
url = "https://api.yelp.com/v3/businesses/search"

# Define headers and parameters for Yelp API (Italian restaurants in Fort Worth)
headers = {"Authorization": f"Bearer {API_KEY}"}
params = {"term": "Italian", "location": "Fort Worth, TX", "categories": "restaurants", "limit": 50}

# Request Yelp API for restaurants
response = requests.get(url, headers=headers, params=params)
data = response.json()

# Extract restaurant information
restaurants = []
for business in data["businesses"]:
    name = business["name"]
    lat = business["coordinates"]["latitude"]
    lon = business["coordinates"]["longitude"]
    address = ", ".join(business["location"]["display_address"])
    restaurants.append((name, lat, lon, address))

# Assuming df is the DataFrame containing crime data with 'latitude' and 'longitude' columns
# Create a base map centered on Fort Worth
fort_worth_map = folium.Map(location=[32.7555, -97.3308], zoom_start=12)

# Create a MarkerCluster for crime locations
marker_cluster = MarkerCluster().add_to(fort_worth_map)

# Plot crime locations (assuming df has the necessary data)
for _, row in df.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=row["block_address"]
    ).add_to(marker_cluster)

# Plot restaurant locations as individual markers
for restaurant in restaurants:
    folium.Marker(
        location=[restaurant[1], restaurant[2]],
        popup=f"{restaurant[0]}<br>{restaurant[3]}",
        icon=folium.Icon(color="red", icon="cutlery")
    ).add_to(fort_worth_map)

# Show the map with both crime and restaurant markers
fort_worth_map


In [213]:
API_KEY = "kOD-SoCJviK3JT-7j-p6mb5-k6b8yISrvdAIhjPfzFTz4TA_ivqQy0SO-t5nk1COe-jY8zdRL63GlYVPoleEP3Y3JM67iLl0GWW_gdRdXvqWB36BUBOROZMpQjCwZ3Yx"
url = "https://api.yelp.com/v3/businesses/search"

# Define headers and parameters for Yelp API (Italian restaurants in Fort Worth)
headers = {"Authorization": f"Bearer {API_KEY}"}
params_wholeFoods = {"term": "Whole Foods Market", "location": "Fort Worth, TX", "limit": 50}

# Request Yelp API for restaurants
response = requests.get(url, headers=headers, params=params_wholeFoods)
data_wholeFoods = response.json()

wholeFoods = []
for business in data_wholeFoods.get("businesses", []):  # Ensure "businesses" key exists
    if "Whole Foods Market" in business["name"]:  # Filter exact store name
        name = business["name"]
        lat = business["coordinates"]["latitude"]
        lon = business["coordinates"]["longitude"]
        rating = business["rating"]
        address = ", ".join(business["location"]["display_address"])
        wholeFoods.append((name, lat, lon, rating, address))

# Check if data was fetched correctly
print(f"Fetched {len(wholeFoods)} wholeFoods.")

for store in wholeFoods:
    print(f"Name: {store[0]}, Address: {store[4]}, Rating: {store[3]}")



Fetched 3 wholeFoods.
Name: Whole Foods Market, Address: 3720 Vision Dr, Fort Worth, TX 76109, Rating: 2.8
Name: Whole Foods Market, Address: 4801 Colleyvile Blvd, Colleyville, TX 76034, Rating: 3.5
Name: Whole Foods Market, Address: 801 E Lamar, Arlington, TX 76011, Rating: 3.6


In [227]:
fort_worth_wholeFoods = folium.Map(location=[32.7555, -97.3308], zoom_start=12)

# Add markers for each restaurant
for name, lat, lon, rating, address in wholeFoods:
    popup_text = f"<b>{name}</b><br>Rating: {rating} ⭐<br>{address}"
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color="green", icon="leaf", prefix="fa"),
    ).add_to(fort_worth_wholeFoods)


# Display the map
fort_worth_wholeFoods


In [229]:
API_KEY = "kOD-SoCJviK3JT-7j-p6mb5-k6b8yISrvdAIhjPfzFTz4TA_ivqQy0SO-t5nk1COe-jY8zdRL63GlYVPoleEP3Y3JM67iLl0GWW_gdRdXvqWB36BUBOROZMpQjCwZ3Yx"
url = "https://api.yelp.com/v3/businesses/search"

# Define headers and parameters for Yelp API (Italian restaurants in Fort Worth)
headers = {"Authorization": f"Bearer {API_KEY}"}
params_costco = {"term": "Costco", "location": "Fort Worth, TX", "limit": 50}

# Request Yelp API for restaurants
response = requests.get(url, headers=headers, params=params_costco)
data_costco = response.json()

costco = []
for business in data_costco.get("businesses", []):  # Ensure "businesses" key exists
    if "Costco" in business["name"]:  # Filter exact store name
        name = business["name"]
        lat = business["coordinates"]["latitude"]
        lon = business["coordinates"]["longitude"]
        rating = business["rating"]
        address = ", ".join(business["location"]["display_address"])
        costco.append((name, lat, lon, rating, address))

# Check if data was fetched correctly
print(f"Fetched {len(costco)} costco.")

for store in costco:
    print(f"Name: {store[0]}, Address: {store[4]}, Rating: {store[3]}")



Fetched 3 costco.
Name: Costco Warehouse, Address: 5300 Overton Ridge Blvd, Fort Worth, TX 76132, Rating: 3.2
Name: Costco Wholesale, Address: 8900 Tehama Ridge Pkwy, Fort Worth, TX 76177, Rating: 3.6
Name: Costco Wholesale, Address: 600 W Arbrook Blvd, Arlington, TX 76014, Rating: 3.4


In [247]:
fort_worth_costco = folium.Map(location=[32.7555, -97.3308], zoom_start=12)

# Add markers for each restaurant
for name, lat, lon, rating, address in costco:
    popup_text = f"<b>{name}</b><br>Rating: {rating} ⭐<br>{address}"
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color="purple", icon="basket-shopping", prefix="fa"),
    ).add_to(fort_worth_costco)


# Display the map
fort_worth_costco


In [265]:

# Load CSV file
df_income = pd.read_csv("fort_worth_income.csv")  # Replace with your file path

# Show first 5 rows
df_income.head()

print(len(df_income.columns))

457
